In [ ]:
import torch
import pytorch_lightning as pl

from src.render.mesh_renderer import MeshPointsRenderer
from src.models.discriminator import Discriminator
from src.models.generator import Generator

class GAN(pl.LightningModule):
    
    def __init__(self, hparams):
        super().__init__()
        self.hparams = hparams
        #self.automatic_optimization = False
        self.mean = sum(hparams.image_mean) / len(hparams.image_mean)
        self.std = sum(hparams.image_std) / len(hparams.image_std)
        
        self.G = Generator(hparams)
        self.D = Discriminator(hparams)                
        self.R = MeshPointsRenderer(hparams)     
        self.edge_loss = EdgeLoss(hparams)
        
     
    def forward(self, points):
        return self.G(points)
    
    def adversarial_loss(self, y_hat, y):
        return F.mse_loss(y_hat, y)
    
    def training_step(self, batch, batch_idx, optimizer_idx):        
        img_patch = batch['image']
        blueprint = batch['blueprint']
        
        self.R.setup(points.device)        
        
        # train generator
        if optimizer_idx == 0:
            
            return OrderedDict({ 'loss': g_loss, })

        # train discriminator
        if optimizer_idx == 1:            
            
            self.log("loss/d_loss", d_loss, on_step=True)            
            return OrderedDict({ 'loss': d_loss, })

    
    def configure_optimizers(self):
        lr_g = self.hparams.lr_g
        lr_d = self.hparams.lr_d
        b1 = self.hparams.beta1
        b2 = self.hparams.beta2      
        opt_g = torch.optim.Adam(self.G.parameters(), 
                                 lr=lr_g, betas=(b1, b2))
        opt_d = torch.optim.Adam(self.D.parameters(), 
                                 lr=lr_d, betas=(b1, b2))
        return [opt_g, opt_d], []